In [7]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

In [8]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'amazon-beauty/large_merged_data.csv')
print(f'Data path: {data_path}')

# zero shot save path
# CF_ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/CF_large_predictions_zero_shot.csv')
# print(f'Zero shot save path: {CF_ZERO_SHOT_SAVE_PATH}')

# few shot save path
CF_FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/CF_large_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {CF_FEW_SHOT_1_OBS_SAVE_PATH}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/large_merged_data.csv
Few shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/CF_large_1_test_predictions_few_shot.csv


In [9]:
# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9767 entries, 0 to 9766
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rating          9767 non-null   float64
 1   verified        9767 non-null   bool   
 2   reviewTime      9767 non-null   object 
 3   reviewerID      9767 non-null   object 
 4   asin            9767 non-null   object 
 5   reviewerName    9767 non-null   object 
 6   reviewText      9759 non-null   object 
 7   summary         9759 non-null   object 
 8   unixReviewTime  9767 non-null   object 
 9   vote            1487 non-null   object 
 10  style           6768 non-null   object 
 11  category        9767 non-null   object 
 12  tech1           2 non-null      object 
 13  description     9767 non-null   object 
 14  fit             0 non-null      float64
 15  title           9767 non-null   object 
 16  also_buy        9767 non-null   object 
 17  tech2           0 non-null      f

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
0,1.0,False,2015-08-25,A2RYSCZOPEXOCQ,9790787006,The Cat Next Door,"I use a lot of perfume, I go through a new bot...",This is not going to be my favorite scent.,2015-08-25,NaN,...,NaN,Jenna Jameson,[],298.0,"['B00357FTX8', 'B01NBID7FJ', 'B0017JT658']","{'Shipping Weight:': '12.8 ounces (', 'ASIN: '...",All Beauty,NaN,NaN,13.85
1,5.0,False,2001-06-08,A141OPVE376YFI,B000050B65,Paul G.,"First, a little background. I've switched bet...","Finally, a razor that lives up to the ads",2001-06-08,81,...,NaN,Norelco,[],2.0,"['B01B1O9DOM', 'B00JITDVD2', 'B01KXV16DK', 'B0...",{},All Beauty,NaN,NaN,NaN
2,5.0,False,2001-06-08,A141OPVE376YFI,B000050B65,Paul G.,"First, a little background. I've switched bet...","Finally, a razor that lives up to the ads",2001-06-08,81,...,NaN,Norelco,[],2.0,"['B01B1O9DOM', 'B00JITDVD2', 'B01KXV16DK', 'B0...",{},All Beauty,NaN,NaN,NaN
3,5.0,True,2008-07-25,A1TVTDKNMSQ7XU,B000050B6B,Grandpa Pipes,I've had many Norelco razors in my 50 years of...,Just like new.....,2008-07-25,NaN,...,NaN,Philips Norelco,[],148.0,"['B001IA0PCY', 'B00196W5S4', 'B004URZADG', 'B0...",{'\n Product Dimensions: \n ': '5.1 x 0....,All Beauty,NaN,NaN,64.50
4,5.0,True,2008-07-25,A1TVTDKNMSQ7XU,B000050B6B,Grandpa Pipes,I've had many Norelco razors in my 50 years of...,Just like new.....,2008-07-25,NaN,...,NaN,Philips Norelco,[],148.0,"['B001IA0PCY', 'B00196W5S4', 'B004URZADG', 'B0...",{'\n Product Dimensions: \n ': '5.1 x 0....,All Beauty,NaN,NaN,64.50


In [10]:
# Function to create a user-item interaction matrix
def create_interaction_matrix(df, user_col, item_col, rating_col, threshold=0):
    """
    Create the user-item interactions matrix.
    
    :param df: DataFrame containing user-item interactions.
    :param user_col: Name of the user column.
    :param item_col: Name of the item column.
    :param rating_col: Name of the rating column.
    :param threshold: Minimum rating to consider.
    :return: A sparse user-item interaction matrix and mapper dictionaries.
    """
    interactions = df.groupby([user_col, item_col])[rating_col] \
                     .sum().unstack().reset_index(). \
                     fillna(0).set_index(user_col)
    interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    
    user_mapper = dict(zip(np.unique(df[user_col]), list(range(df[user_col].nunique()))))
    item_mapper = dict(zip(np.unique(df[item_col]), list(range(df[item_col].nunique()))))

    user_inv_mapper = dict(zip(list(range(df[user_col].nunique())), np.unique(df[user_col])))
    item_inv_mapper = dict(zip(list(range(df[item_col].nunique())), np.unique(df[item_col])))

    user_index = [user_mapper[i] for i in interactions.index]
    item_index = [item_mapper[i] for i in interactions.columns]

    X = csr_matrix(interactions.values)

    return X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper

# Function to fit the kNN model
def fit_knn_model(interaction_matrix, n_neighbors=4):
    """
    Fit the k-Nearest Neighbors model.
    
    :param interaction_matrix: User-item interaction matrix.
    :param n_neighbors: Number of neighbors to consider.
    :return: Trained kNN model.
    """
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_neighbors, n_jobs=-1)
    model_knn.fit(interaction_matrix)
    return model_knn

# Function to find similar users and recommend items
def recommend_items(user_id, interaction_matrix, user_mapper, item_inv_mapper, model_knn, n_recommendations=4):
    """
    Recommend items for a given user.
    
    :param user_id: User ID for whom to make recommendations.
    :param interaction_matrix: User-item interaction matrix.
    :param user_mapper: Dictionary mapping user ID to user index.
    :param item_inv_mapper: Dictionary mapping item index to item ID.
    :param model_knn: Trained kNN model.
    :param n_recommendations: Number of recommendations to make.
    :return: List of recommended item IDs.
    """
    user_idx = user_mapper[user_id]
    distances, indices = model_knn.kneighbors(interaction_matrix[user_idx], n_neighbors=n_recommendations+1)
    
    raw_recommends = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    recommendations = []
    for i, (idx, dist) in enumerate(raw_recommends):
        if idx != user_idx:  # Skip the user itself
            recommendations.append(item_inv_mapper[idx])

    return recommendations


In [11]:
%%time

# Example DataFrame columns
user_col = 'reviewerID'  # Replace with your user column name
item_col = 'asin'    # Replace with your item column name
rating_col = 'rating' # Replace with your rating column name

# Step 1: Create User-Item Interaction Matrix
interaction_matrix, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper = create_interaction_matrix(data, user_col, item_col, rating_col)

# Step 2: Fit the kNN Model
model_knn = fit_knn_model(interaction_matrix)

# Step 3: Make Recommendations for a specific user
user_id = 'ANV9L0JU6BNL'  # Replace with a user ID from your dataset
recommendations = recommend_items(user_id, interaction_matrix, user_mapper, item_inv_mapper, model_knn, n_recommendations=4)

print("Recommended Items:", recommendations)


Recommended Items: ['B016FLEZH6', 'B00P05M0D0', 'B012E93QIW', 'B000NKJIXM']
CPU times: user 868 ms, sys: 36.3 ms, total: 905 ms
Wall time: 902 ms


# Collaborative filtering model to predict ratings - User based CF

In [12]:
def predict_ratings_with_collaborative_filtering_and_save(data, interaction_matrix, user_mapper, item_mapper, user_inv_mapper, model_knn,
                                                          columns_for_training, columns_for_prediction,
                                                          user_column_name='reviewerID', title_column_name='title', asin_column_name='asin',
                                                          obs_per_user=None, pause_every_n_users=PAUSE_EVERY_N_USERS, sleep_time=SLEEP_TIME,
                                                          save_path='../../data/amazon-beauty/similar_users_predictions.csv'):
    predicted_ratings = []
    actual_ratings = []
    users = data[user_column_name].unique()

    for idx, user_id in enumerate(users):
        user_data = data[data[user_column_name] == user_id]

        if len(user_data) >= 5:
            test_data = user_data.sample(obs_per_user if obs_per_user else 1, random_state=RANDOM_STATE)
            remaining_data = user_data[~user_data.index.isin(test_data.index)]

            for test_idx, test_row in test_data.iterrows():
                user_idx = user_mapper[user_id]
                distances, indices = model_knn.kneighbors(interaction_matrix[user_idx], n_neighbors=5)

                training_data = pd.DataFrame()
                similar_users = []
                for idx in indices.flatten():
                    similar_user_id = user_inv_mapper[idx]
                    if similar_user_id != user_id:
                        similar_users.append(similar_user_id)
                        similar_user_data = data[data[user_column_name] == similar_user_id]
                        training_data = pd.concat([training_data, similar_user_data], ignore_index=True)

                # Log similar users
                print(f"Similar users for {user_id}: {similar_users}")

                prediction_data = {col: test_row[col] for col in columns_for_prediction if col != 'rating'}
                combined_text = generate_combined_text_for_prediction(columns_for_prediction, *prediction_data.values())

                rating_history_str = ', '.join([f"{row[columns_for_training[0]]} ({row['rating']} stars)" for _, row in training_data.iterrows()])
                predicted_rating = predict_rating_combined_ChatCompletion(combined_text, rating_history=rating_history_str, use_few_shot=True)

                predicted_ratings.append(predicted_rating)
                actual_ratings.append(test_row['rating'])

                print(f"Processing user {idx + 1}/{len(users)}, item {test_idx + 1}/{len(test_data)}")
                print(f"User {user_id}:")
                print(f"Rating History for Prediction: {rating_history_str}")
                print(f"Predicted Item: {test_row[title_column_name]}")
                print(f"Predicted Rating: {predicted_rating} stars")
                print("\n----------------\n")

            if (idx + 1) % pause_every_n_users == 0:
                print(f"Processed {idx + 1} users. Pausing for {sleep_time} seconds...")
                time.sleep(sleep_time)

    predicted_ratings_df = pd.DataFrame({'predicted_rating': predicted_ratings, 'actual_rating': actual_ratings})
    predicted_ratings_df.to_csv(save_path, index=False)
    print(f"Predictions saved to {save_path}")



In [13]:
%%time

interaction_matrix, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper = create_interaction_matrix(data, user_col, item_col, rating_col)
model_knn = fit_knn_model(interaction_matrix)

predict_ratings_with_collaborative_filtering_and_save(data, interaction_matrix, user_mapper, item_mapper, user_inv_mapper, model_knn,
                                                      ITEM_SIDE + USER_SIDE + INTERACTION_SIDE, ITEM_SIDE,
                                                      user_column_name='reviewerID', title_column_name='title', asin_column_name='asin',
                                                      obs_per_user=1, save_path=CF_FEW_SHOT_1_OBS_SAVE_PATH)


Similar users for A2RYSCZOPEXOCQ: ['A2NOIGZ6VV86IB', 'A6P370Q9YB27F', 'A2MBLVMVDLHK1L', 'ATV6UWVB04G2S']
System Fingerprint: fp_eeff13170a
Unexpected response for the provided details: Based on the product details provided, the rating for the Kordon Oasis Bell Bottle 8oz might be influenced by factors such as its suitability for various small animals, the quality and durability of the bottle, ease of use, and the overall value for the price. Additionally, customer satisfaction with the product's performance and longevity could also impact the rating. Given the lack of specific category and technical details, the rating might also be influenced by the brand reputation and customer service.
Processing user 1537/1608, item 8724/1
User A2RYSCZOPEXOCQ:
Rating History for Prediction: B00021DJ32 (5.0 stars), B00021DJ32 (5.0 stars), B006WYJM8Y (5.0 stars), B00NT0AR7E (4.0 stars), B01DEDYWD0 (4.0 stars), B00021KD2C (4.0 stars), B00021KD2C (4.0 stars), B00NT0AR7E (5.0 stars), B00UT0AFFG (5.0 sta

In [ ]:
# Evaluate Few-shot Model
evaluate_model_predictions_rmse_mae(
    data_path=CF_FEW_SHOT_1_OBS_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)